In [5]:
ls

kaggle.json  sample_data/


In [0]:
!pip install -q kaggle

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/


In [0]:
!chmod 600 ~/.kaggle/kaggle.json

In [9]:
!kaggle competitions download -c jigsaw-toxic-comment-classification-challenge

  0% 0.00/1.39M [00:00<?, ?B/s]
100% 1.39M/1.39M [00:00<00:00, 97.4MB/s]
 77% 18.0M/23.4M [00:00<00:00, 49.3MB/s]
100% 23.4M/23.4M [00:00<00:00, 93.0MB/s]
 65% 17.0M/26.3M [00:00<00:00, 30.2MB/s]
100% 26.3M/26.3M [00:00<00:00, 41.5MB/s]
  0% 0.00/1.46M [00:00<?, ?B/s]
100% 1.46M/1.46M [00:00<00:00, 98.7MB/s]


In [10]:
!pip install ktrain

     |████████████████████████████████| 81kB 3.8MB/s 
     |████████████████████████████████| 317kB 10.2MB/s 
     |████████████████████████████████| 112kB 46.2MB/s 
     |████████████████████████████████| 1.0MB 37.6MB/s 
     |████████████████████████████████| 204kB 45.4MB/s 
  Created wheel for ktrain: filename=ktrain-0.4.2-cp36-none-any.whl size=96200 sha256=ca74d1e97c940f9b0fb662ea00a62e3bc9e45430d72fff1fbc0f4204f4569287
  Stored in directory: /root/.cache/pip/wheels/ee/37/c7/ff6ed285bd4b6457fbc87487793c49ab3cd0220cee3e1dc331
  Created wheel for keras-bert: filename=keras_bert-0.80.0-cp36-none-any.whl size=37923 sha256=32275a1227e7b5e8e2e7a213e01f1e96d87fe549471a207eef77d3eccde86ab5
  Stored in directory: /root/.cache/pip/wheels/63/dc/87/3260cb91f3aa32c0f85c5375429a30c8fd988bbb48f5ee21b0
  Created wheel for seqeval: filename=seqeval-0.0.12-cp36-none-any.whl size=7424 sha256=2f5fefc55a98efce43950b9b6003e94d9495769c31967a98f2f8b96c74b44ed7
  Stored in directory: /root/.cache/pip/whee

In [12]:
!unzip train.csv.zip

Archive:  train.csv.zip
  inflating: train.csv               


In [0]:
import numpy as np
import pandas as pd

In [0]:
data=pd.read_csv('/content/train.csv')

In [15]:
data.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [16]:
data.shape

(159571, 8)

In [17]:
!unzip test.csv.zip


Archive:  test.csv.zip
  inflating: test.csv                


In [0]:
data1=pd.read_csv('/content/test.csv')

In [19]:
data1.head()

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [20]:
data1.shape

(153164, 2)

In [22]:
!unzip test_labels.csv.zip

Archive:  test_labels.csv.zip
  inflating: test_labels.csv         


In [0]:
data2=pd.read_csv("/content/test_labels.csv")

In [24]:
data2.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,-1,-1,-1,-1,-1,-1
1,0000247867823ef7,-1,-1,-1,-1,-1,-1
2,00013b17ad220c46,-1,-1,-1,-1,-1,-1
3,00017563c3f7919a,-1,-1,-1,-1,-1,-1
4,00017695ad8997eb,-1,-1,-1,-1,-1,-1


In [25]:
import ktrain
from ktrain import text

Using TensorFlow backend.


using Keras version: 2.2.4


In [27]:
path='/content/train.csv'
words=50000
length=150
(x_train,y_train),(x_test,y_teat), preproc=text.texts_from_csv(path,'comment_text',label_columns = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"],val_filepath=None,max_features=words,maxlen=length,ngram_range=1)

detected encoding: utf-8 (if wrong, set manually)
language: en
Word Counts: 197754
Nrows: 143613
143613 train sequences
Average train sequence length: 66
x_train shape: (143613,150)
y_train shape: (143613,6)
15958 test sequences
Average test sequence length: 65
x_test shape: (15958,150)
y_test shape: (15958,6)


In [29]:
model=text.text_classifier('bigru', (x_train,y_train), preproc=preproc)

Is Multi-Label? True
compiling word ID features...
maxlen is 150
processing pretrained word vectors...
Loading pretrained word vectors...this may take a few moments...
downloading pretrained word vectors (~1.5G) ...
[██████████████████████████████████████████████████]
extracting pretrained pretrained word vectors...
done.

cleanup downloaded zip...
done.

Done.
done.


In [0]:
learner=ktrain.get_learner(model, train_data=(x_train,y_train), val_data=(x_test,y_teat))

In [32]:
learner.lr_find()

simulating training for different learning rates... this may take a few moments...
Epoch 1/1024
   576/143613 [..............................] - ETA: 57:57 - loss: 0.7452 - acc: 0.3134

In [35]:
learner.autofit(0.001, 1)



begin training using triangular learning rate policy with max lr of 0.001...
Train on 143613 samples, validate on 15958 samples
Epoch 1/1
143613/143613 [==============================] - 2823s 20ms/step - loss: 0.0586 - acc: 0.9801 - val_loss: 0.0406 - val_acc: 0.9846


In [37]:
from sklearn.metrics import roc_auc_score
y_pred = learner.model.predict(x_test, verbose=0)
score = roc_auc_score(y_teat, y_pred)
print("\n ROC-AUC score: %.6f \n" % (score))




 ROC-AUC score: 0.984993 



In [0]:
predictor = ktrain.get_predictor(learner.model, preproc)



In [39]:
predictor.predict(["If you don't stop immediately, I will kill you."])

[[('toxic', 0.8659534),
  ('severe_toxic', 0.018227737),
  ('obscene', 0.10012532),
  ('threat', 0.08051027),
  ('insult', 0.22765163),
  ('identity_hate', 0.014658908)]]

In [40]:
predictor.predict(["Okay - I'm calling it a night. See you tomorrow."])

[[('toxic', 0.010844667),
  ('severe_toxic', 0.0005329362),
  ('obscene', 0.0017846359),
  ('threat', 0.0011022425),
  ('insult', 0.003713013),
  ('identity_hate', 0.0006030426)]]